In [44]:
using Revise
using JSON, LinearAlgebra
using Plots, Printf
using Random
using LineSearches, Optim
using DRSOM
using LaTeXStrings
using LazyStack
using ADNLPModels
using SNLOpt

json_data = JSON.parsefile("instances_cyl/formatted_instance4.json")


Dict{String, Any} with 8 entries:
  "anchors"                => Any[Dict{String, Any}("x"=>1.91301, "y"=>8.97606)…
  "d_ij"                   => Dict{String, Any}("8_29"=>4.72285, "25_11"=>7.845…
  "total_edges"            => Any[Dict{String, Any}("start"=>20, "end"=>4), Dic…
  "N_x"                    => Any[Any[24, 12], Any[19, 13], Any[8, 9], Any[17, …
  "d_ik"                   => Dict{String, Any}("23_4"=>4.02519, "21_3"=>11.088…
  "N_a"                    => Any[Any[6, 1], Any[6, 2], Any[6, 3], Any[6, 4], A…
  "total_sensor_positions" => Any[Dict{String, Any}("x"=>1.91301, "y"=>8.97606)…
  "loss"                   => 5.77789e-5

##### anchors, rigid points, random points

In [45]:
anchors = [(pos["x"], pos["y"]) for pos in json_data["anchors"]]
total_sensor_positions = [(pos["x"], pos["y"]) for pos in json_data["total_sensor_positions"]]
total_edges = [(edge["start"], edge["end"]) for edge in json_data["total_edges"]]
d_ij = Dict((parse(Int, split(key, "_")[1]), parse(Int, split(key, "_")[2])) => value for (key, value) in json_data["d_ij"])
d_ik = Dict((parse(Int, split(key, "_")[1]), parse(Int, split(key, "_")[2])) => value for (key, value) in json_data["d_ik"])
posmat = hcat([[i...] for i in total_sensor_positions]...)

2×30 Matrix{Float64}:
 1.91301  5.39162  0.501416  7.68306  …  6.82497  2.3449   5.49024  7.5757
 8.97606  8.83896  1.54933   5.00119     8.78581  6.58107  5.62457  8.53347

In [46]:
total_sensor_positions

30-element Vector{Tuple{Float64, Float64}}:
 (1.9130133690227402, 8.976058183223675)
 (5.391618097638133, 8.838957518886824)
 (0.5014163739287192, 1.5493255840631204)
 (7.683063186870936, 5.0011889152709355)
 (9.315698798818993, 5.766109602786653)
 (3.402085261208784, 2.6353414227911545)
 (8.237079586938847, 4.79167905670656)
 (1.2954173956015325, 7.794540366477978)
 (6.83830458234855, 4.824332821967554)
 (3.883966790625651, 6.571756668281142)
 ⋮
 (2.947909660558241, 1.6171854668897367)
 (5.4695233678109245, 8.36308817962502)
 (2.053646121968203, 4.02782223418598)
 (7.753407867260637, 6.009239287698506)
 (2.6939220122075556, 4.5383246098447785)
 (6.824974285759029, 8.78581449409668)
 (2.344897145969963, 6.581074086527231)
 (5.490239060400961, 5.624574870974851)
 (7.575702560094564, 8.533467335473704)

In [47]:
@assert (norm(posmat[:, 25] - posmat[:, 2]) - d_ik[(25,2)]) ≈ 0
PP = hcat([[i...] for i in total_sensor_positions]...)
n = size(PP, 2)
m = length(anchors)
nf = 0.0
idxanchors = [1:m...]
nonanchors = [m+1:n...]
get_type(i,j) = ((i in idxanchors) || (j in idxanchors)) ? 'a' : 'x'

# set to SDP style, last m sensors are anchors
sete = [(get_type(i,j), sort([n+1-i,n+1-j])) for ((i, j), _) in Dict(d_ik..., d_ij...) if i <= n && j <= n]
pp = posmat[:, [nonanchors..., idxanchors...]]

2×30 Matrix{Float64}:
 3.40209  8.23708  1.29542  6.8383   …  5.39162  0.501416  7.68306  9.3157
 2.63534  4.79168  7.79454  4.82433     8.83896  1.54933   5.00119  5.76611

In [48]:
Nxd = SNLOpt.create_neighborhood(n, m, pp, nf, sete)

201-element Vector{Neighbor}:
 Neighbor((16, 26), 'a', [-1.9130133690227402, -8.976058183223675, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 6.8490301394632835, 6.8490301394632835)
 Neighbor((8, 29), 'a', [-7.683063186870936, -5.0011889152709355, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 8.919269883249141, 8.919269883249141)
 Neighbor((20, 29), 'a', [-7.683063186870936, -5.0011889152709355, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1.0105018195978137, 1.0105018195978137)
 Neighbor((24, 30), 'a', [-9.315698798818993, -5.766109602786653, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], 3.828077100917168, 3.828077100917168)
 Neighbor((6, 30), 'a', [-9.315698798818993, -5.766109602786653, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 8.79679073420

In [49]:
Nxdview = Dict(nx.edge => nx for nx in Nxd)
@info("neighborhood (limited degree) created with size: $(length(Nxd))")
# limited batch loss, gradient, and Hessian
fd(x) = loss(x, pp, Nxdview, nothing, keys(Nxdview))
gd(x) = g(x, pp, Nxdview, nothing, keys(Nxdview))
Hd(x) = H(x, pp, Nxdview, nothing, keys(Nxdview))
hvp(x, v, w) = SNLOpt.hvp!(x, v, w, pp, Nxdview, nothing, keys(Nxdview))

Xv = rand(Float64, (2, n - m))
x0 = reshape(Xv, length(Xv))
function ADNLPModels.gradient!(adbackend::SNL.NewADGradient, w, f, x)
    copyto!(w, gd(x))
end
@time begin
    @info("create a SNL for NLPModels.jl API")
    nlp = ADNLPModel(fd, copy(x0), gradient_backend=SNL.NewADGradient)
end

┌ Info: neighborhood (limited degree) created with size: 201
└ @ Main /Users/brent/workspace/SNL.jl/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X24sZmlsZQ==.jl:2
┌ Info: create a SNL for NLPModels.jl API
└ @ Main /Users/brent/workspace/SNL.jl/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X24sZmlsZQ==.jl:15


  0.525589 seconds (26.70 k allocations: 2.910 MiB, 99.81% compilation time: <1% of which was recompilation)


ADNLPModel - Model with automatic differentiation backend ADModelBackend{
  SNLOpt.SNL.NewADGradient,
  ForwardDiffADHvprod,
  EmptyADbackend,
  EmptyADbackend,
  EmptyADbackend,
  SparseADHessian,
  EmptyADbackend,
}
  Problem name: Generic
   All variables: ████████████████████ 50     All constraints: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            free: ████████████████████ 50                free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: ( 82.20% sparsity)   227             linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                              

In [50]:
r = Dict{String, Any}()

Dict{String, Any}()

In [51]:
r[@sprintf("DRSOM")] = DRSOM2(name=Symbol("DRSOM"))(;
    x0=copy(x0), f=fd, g=gd,
    maxiter=10000, freq=10,
    maxtime=10000
)
r[@sprintf("Newton-TR")] = optim_to_result(Optim.optimize(
            fd, gd, Hd, x0,
            NewtonTrustRegion(;
            ),
            SNLOpt.somoptions;
            inplace=false
        ), "Newton-TR"
)

-----------------------------------------------------------------------------------------------
                           The Dimension-Reduced Second-Order Method                           
                    (c) Chuwen Zhang, Yinyu Ye, Cardinal Operations (2022)                    
-----------------------------------------------------------------------------------------------
 algorithm alias       := DRSOM2
 algorithm description := DRSOM with gradient and momentum
 inner iteration limit := 20
 oracle (first-order)  := direct => provided
 oracle (second-order) := direct => use interpolation
-----------------------------------------------------------------------------------------------
    k |          f |       α₁ |       α₂ |       Δ |    |∇f| |       λ |      ρ | kₜ |      t 
    1 | +1.827e+05 | +3.2e-04 | +0.0e+00 | 3.2e-04 | 4.0e+04 | 1.0e-06 | +6e-01 |  2 |    0.1 
   10 | +3.304e+01 | +2.7e-01 | +1.7e-01 | 3.1e-01 | 4.0e+02 | 5.8e-01 | +1e+00 |  1 |    0.2 
   20 | +2.071e-

Result<Newton-TR>


In [52]:
pgfplotsx()

push!(PGFPlotsX.CUSTOM_PREAMBLE, raw"\usepackage{amssymb}")
plot_function_value(n, m, r; smoothing=false, smoothing_range=15, metric=:ϵ)
fig, comments = plot_realization(n, m, pp, Nxd, r; heval=false)
for k in comments
    println(k)
end

plotting results
plotting results


┌ Info: 
│   v.state = DRSOM.DRSOMState{Float64, Vector{Float64}, Matrix{Float64}, Vector{Float64}}([3.4020852611877275, 2.635341422793213, 8.23707958692974, 4.79167905678719, 1.295417395557767, 7.794540366368487, 6.838304582319229, 4.824332821978355, 3.883966790639708, 6.571756668221751, 2.5720275334526033, 0.11753234065461286, 5.523181345480034, 1.2154821755905911, 0.1349828830689914, 0.24936200575161055, 5.517081572655483, 3.34058646241811, 0.5562698416487256, 3.085613861467423, 0.8005443995427614, 1.1854755836823818, 8.721092125795412, 5.955132217892538, 4.499252270348127, 5.7426565386252015, 2.565150314620041, 8.925078525560101, 9.524596387594825, 3.1708566864275776, 8.761979185169773, 8.946374840421793, 2.9479096603179165, 1.6171854669432126, 5.469523367897761, 8.363088179564519, 2.0536461219835833, 4.027822234126408, 7.753407867271243, 6.009239287685575, 2.693922012171593, 4.538324609981514, 6.8249742857292945, 8.785814494156368, 2.3448971459358754, 6.581074086508383, 5.49023906